In [57]:
import pythoncom
import win32com.client as winAPI
import time

In [58]:
STAND_BY = 0
RECEIVED = 1


class XASessionEvents:
    login_state = STAND_BY

    def OnLogin(self, code, msg):
        XASessionEvents.login_state = RECEIVED
        print(msg)

    def OnDisconnect(self, code, msg):
        pass


class XAQueryEvents:
    query_state = STAND_BY

    def OnReceiveData(self, code):
        XAQueryEvents.query_state = RECEIVED

    def OnReceiveMessage(self, error, nMessageCode, szMessage):
        print(szMessage)

In [59]:
import datetime
import time

SERVER_PORT = 20001
SHOW_CERTIFICATE_ERROR_DIALOG = False
REPEATED_DATA_QUERY = 1
TRANSACTION_REQUEST_EXCESS = -21
TODAY = datetime.datetime.now().strftime('%Y%m%d')

if __name__ == "__main__":
    id = "ssuyan"
    password = "tndus1!!"
    certificate_password = "qkrtndus1!!"
    xa_session = winAPI.DispatchWithEvents("XA_Session.XASession", XASessionEvents)

    # demo.ebestsec.co.kr => 모의투자 
    # hts.ebestsec.co.kr => 실투자
    xa_session.ConnectServer("hts.ebestsec.co.kr", SERVER_PORT)
    xa_session.Login(id, password, certificate_password, SERVER_PORT, SHOW_CERTIFICATE_ERROR_DIALOG)

    while XASessionEvents.login_state is STAND_BY:
        pythoncom.PumpWaitingMessages()
    XASessionEvents.login_state = STAND_BY

로그인 성공


In [60]:
TR = "t8430"
xa_query = winAPI.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEvents)
xa_query.ResFileName = "C:\\eBEST\\xingAPI\\Res\\" + TR + ".res"

xa_query.SetFieldData("t8430InBlock", "gubun", 0, 1)

while True:
    ret = xa_query.Request(False)
    """ Receiving error message, keep requesting until accepted """
    if ret is TRANSACTION_REQUEST_EXCESS:  # -34
        time.sleep(0.8)
    else:
        break
""" Wait window's event message """
while XAQueryEvents.query_state is STAND_BY:
    pythoncom.PumpWaitingMessages()
XAQueryEvents.query_state = STAND_BY

정상적으로 조회가 완료되었습니다.


In [63]:
import win32com.client
import pythoncom
import time

class XQuery_t1101:
    def __init__(self):
        super().__init__()
        self.is_data_received = False

    def OnReceiveData(self, tr_code):
        """
        이베스트 서버에서 ReceiveData 이벤트 받으면 실행되는 event handler
        """
        self.is_data_received = True
        name = self.GetFieldData("t1101OutBlock", "hname", 0)
        hotime = self.GetFieldData("t1101OutBlock", "hotime", 0)
        price = self.GetFieldData("t1101OutBlock", "price", 0)
        open = self.GetFieldData("t1101OutBlock", "open", 0)
        high = self.GetFieldData("t1101OutBlock", "high", 0)
        volume = self.GetFieldData("t1101OutBlock", "volume", 0)
        offerrem10 = self.GetFieldData("t1101OutBlock", "offerrem10", 0)
        bidrem10 = self.GetFieldData("t1101OutBlock", "bidrem10", 0)
        offer = self.GetFieldData("t1101OutBlock", "offer", 0)
        bid = self.GetFieldData("t1101OutBlock", "bid", 0)
        print("종목;{0},수신시간;{1}, 현재가;{2},시가;{3},고가;{4},누적거래량;{5},매도호가수량10;{6},매수호가수량10;{7},매도호가수량합;{8},매수호가수량합;{9}".format(name, hotime, price, open, high, volume,offerrem10,bidrem10,offer,bid))
        print("TR code => {0}".format(tr_code))

    def request(self):
    # 매수 매도 체결 수량 컬럼추가

        """
        이베스트 서버에 일회성 TR data 요청함.
        """
        self.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t1101.res"  # RES 파일 등록
        self.SetFieldData("t1101InBlock", "shcode", 0, "012450")  # 한화에어로스페이스
        err_code = self.Request(False)  # data 요청하기 --  연속조회인경우만 True

        if err_code < 0:
            print("error... {0}".format(err_code)) # data 요청하기 --  연속조회인경우만 True

    @classmethod
    def get_instance(cls):
        xq_t1101 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", cls)
        return xq_t1101


class XReal_S3_:
    def __init__(self):
        super().__init__()
        self.count = 0

    def OnReceiveRealData(self, tr_code):  # event handler
        """
        이베스트 서버에서 ReceiveRealData 이벤트 받으면 실행되는 event handler
        """
        self.count += 1
        stockcode = self.GetFieldData("OutBlock", "shcode")
        chetime = self.GetFieldData("OutBlock", "chetime")
        hotime = self.GetFieldData("OutBlock", "hotime")
        price = self.GetFieldData("OutBlock", "price")
        open = self.GetFieldData("OutBlock", "open")
        high = self.GetFieldData("OutBlock", "high")
        volume = self.GetFieldData("OutBlock", "volume")
        offerrem10 = self.GetFieldData("OutBlock", "offerrem10")
        bidrem10 = self.GetFieldData("OutBlock", "bidrem10")
        offer = self.GetFieldData("OutBlock", "offer")
        bid = self.GetFieldData("OutBlock", "bid")
        print(self.count, stockcode, chetime, hotime, price, open, high, volume, offerrem10, bidrem10, offer, bid)

    def start(self):
        """
        이베스트 서버에 실시간 data 요청함.
        """
        self.ResFileName = "C:\\eBEST\\xingAPI\\Res\\S3_.res"  # RES 파일 등록
        self.SetFieldData("InBlock", "shcode", "012450")
        self.AdviseRealData()   # 실시간데이터 요청

    def add_item(self, stockcode):
        # 실시간데이터 요청 종목 추가
        self.SetFieldData("InBlock", "shcode", stockcode)
        self.AdviseRealData()

    def remove_item(self, stockcode):
        # stockcode 종목만 실시간데이터 요청 취소
        self.UnadviseRealDataWithKey(stockcode)

    def end(self):
        self.UnadviseRealData()  # 실시간데이터 요청 모두 취소

    @classmethod
    def get_instance(cls):
        xreal = win32com.client.DispatchWithEvents("XA_DataSet.XAReal", cls)
        return xreal



if __name__ == "__main__":
    def get_single_data():
        xq_t1101 = XQuery_t1101.get_instance()
        xq_t1101.request()

        while xq_t1101.is_data_received == False:
            pythoncom.PumpWaitingMessages()


    def get_real_data():
        xreal = XReal_S3_.get_instance()
        xreal.start()

        while True:
            pythoncom.PumpWaitingMessages()
            xreal.add_item("012450")  # 한화에어로스페이스 주식

    get_single_data()
    get_real_data()
    

종목;한화에어로스페이스,수신시간;11252648, 현재가;138200,시가;142600,고가;142700,누적거래량;303189,매도호가수량10;560,매수호가수량10;1330,매도호가수량합;2785,매수호가수량합;9535


TR code => t1101
1 012450 112533  138300 142600 142700 303190    
2 012450 112544  138300 142600 142700 303197    
3 012450 112544  138300 142600 142700 303198    
4 012450 112545  138300 142600 142700 303200    
5 012450 112548  138300 142600 142700 303201    


KeyboardInterrupt: 